In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
tf.config.experimental_run_functions_eagerly(True)
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D,InputLayer
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Reshape, BatchNormalization
from sklearn.metrics import confusion_matrix

In [ ]:
# Load the data from L1_8000rpm_data/ folder
X_train = pd.read_csv('8000rpm_data_z-score/train_sample.csv')
X_test = pd.read_csv('8000rpm_data_z-score/test_sample.csv')
y_train = pd.read_csv('8000rpm_data_z-score/train_label.csv')
y_test = pd.read_csv('8000rpm_data_z-score/test_label.csv')

In [ ]:
# 构建CNN模型
model = Sequential()
# 为了conv2D，需要将数据reshape成三维
model.add(Reshape((32, 32, 1), input_shape=(1024,)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())  # 添加BN层
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
'''model.add(Conv2D(64, kernel_size=(3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())  # 添加BN层
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))'''
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(9, activation='softmax'))  # 9个需要预测的类别

model.summary()

In [ ]:
# 编译模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

In [ ]:
# 可视化训练过程的 loss 和 accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(model.history.history['loss'], label='train loss')
#plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.title('Loss')
plt.subplot(1, 2, 2)
plt.plot(model.history.history['accuracy'], label='train accuracy')
#plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
# 模型预测
y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
# 计算混淆矩阵
conf_mat = confusion_matrix(y_test_classes, y_pred_classes)

# 绘制混淆矩阵
plt.matshow(conf_mat, cmap='Blues')
for i in range(conf_mat.shape[0]):
    for j in range(conf_mat.shape[1]):
        plt.text(x=j, y=i, s=conf_mat[i, j], va='center', ha='center')

plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# 评估
print("Classification Report:")
print(classification_report(y_pred=y_pred_classes, y_true=y_test_classes))

In [ ]:
# 绘制ROC曲线
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# 将标签二值化
y_test_bin = label_binarize(y_test_classes, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8])
y_pred_bin = label_binarize(y_pred_classes, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8])

# 计算每一类的ROC
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(9):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# 绘制ROC曲线
plt.figure()
lw = 2
colors = ['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'yellow', 'black', 'blue', 'purple']
for i, color in zip(range(9), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()